# chapter4 word2vec 속도 개선

## 4.1 word2vec 개선1
- 입력층의 원핫 표현과 가중치 행렬 $W_{in}$의 곱 계산
- 위 계산이 병목이 되므로 해결해야함
- 단어를 원핫 표현으로 다루기 때문에 어휘 수가 많아지면 원핫 표현의 벡터 크기도 커진다
- Embedding 게층 도입 !

### 4.1.1 Embedding 계층
- 결과적으로 수행하는 일은 단지 행렬의 특정 행을 추출하는 것
- 따라서 가중치 매개변수로부터 '단어 ID에 해당하는 행(벡터)'를 추출하는 계층 = Embedding 계층
- Embedding 계층에 단어 임베딩(분산 표현) 저장

### 4.1.2 Embedding 게층 구현

In [2]:
import numpy as np
W = np.arange(21).reshape(7, 3)
print(W)
print(W[2])
print(W[5])

idx = np.array([1, 0, 3, 0])
print(W[idx])

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]]
[6 7 8]
[15 16 17]
[[ 3  4  5]
 [ 0  1  2]
 [ 9 10 11]
 [ 0  1  2]]


In [4]:
class Embedding:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.idx = None
        
    def forward(self, idx):
        W, = self.params
        self.idx = idx
        out = W[idx]
        return out
    
    def backward(self, dout):
        dW, = self.grads
        dW[...] = 0
        # dW[self.idx] = dout 나쁜예
        
        for i, word_id in enumerate(self.idx):
            dW[word_id] += dout[i]
            
        # np.add.at(dW, self.idx, dout)
        
        return None

## 4.2 word2vec 개선2
- 은닉층과 가중치 행렬 $W_{out}$의 곱 및 Softmax 계층의 계산
- 위 계산이 병목이 되므로 해결해야함
- 네거티브 샘플링 손실함수 도입 !

### 4.2.1 은닉층 이후 계산의 문제점
- softmax를 대신할 가벼운 계산 필요
## $y_k={exp(s_k)\over \sum exp(s_i)}$ ... softmax

### 4.2.2 다중 분류에서 이진 분류로
- 다중분류 문제를 이진분류 문제로 근사하기
- 예) 맥락이 'you'와 'goodbye'일 때, 타깃 단어는 'say'입니까? Yes/No

### 4.2.3 시그모이드 함수와 교차 엔트로피 오차
- 시그모이드 함수
#### $y={1 \over {1+exp(-x)}}$
- 교차 엔트로피 오차
#### $L=-(tlogy+(1-t)log(1-y))$

### 4.2.4 다중 분류에서 이진 분류로 (구현)
- params: 매개변수 저장
- grads: 기울기 저장
- embed: Embedding 계층
- cache: 순전파 시의 계산 결과 잠시 유지

In [5]:
class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None
        
    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)
        self.cache = (h, target_W)
        return out
    
    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)
        
        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

### 4.2.5 네거티브 샘플링
- 긍정적인 예에 대해서는 sigmoid 계층의 출력을 1에 가깝게
- 부정적인 예에 대해서는 sigmoid 계층의 출력을 0에 가깝게
- 긍정적 예를 타깃으로 한 경우 손실을 구하고, 부정적 예를 몇 개 샘플링하여 손실을 구한 후, 각각의 데이터의 손실을 더한 값을 최종손실로 결정

### 4.2.6 네거티브 샘플링의 샘플링 기법
- 부정적 예를 어떻게 샘플링하냐? 말뭉치의 통계 데이터를 기초로 샘플링
- 즉 말뭉치에서 자주 등장하는 단어를 많이 추출하고 드물게 등장하는 단어를 적게 추출
### $P'(w_i)={P(w_i)^{0.75} \over {\sum P(w_j)^{0.75}}}$

In [9]:
import numpy as np

words = ['you', 'say', 'goodbye', 'I', 'hello', '.']
print(np.random.choice(words))

# 5개만 무작위로 샘플링 (중복있음)
print(np.random.choice(words, size=5))

# 5개만 무작위로 샘플링 (중복없음)
print(np.random.choice(words, size=5, replace=False))

# 확률분포에 따라 샘플링
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
print(np.random.choice(words, p=p))

.
['you' 'say' 'hello' 'I' 'hello']
['goodbye' 'hello' 'you' '.' 'say']
you


In [21]:
import sys
sys.path.append('.')
from common.negative_sampling_layer import UnigramSampler

corpus = np.array([0,1,2,3,4,1,2,3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[2 3]
 [0 1]
 [1 2]]


### 4.2.7 네거티브 샘플링 구현
- W: 가중치
- corpus: 말뭉치
- sample_size: 부정적 예의 샘플링 횟수
- UnigramSampler: sampler 저장
- loss_layers[0], embed_dot_layers[0]: 긍정적 예를 다루는 계층
- 긍정적 예의 정답 레이블: 1
- 부정적 예의 정답 레이블: 0

In [17]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

## 4.3 개선판 word2vec 학습

### 4.3.1 CBOW 모델 구현
- vocab_size: 어휘수
- hidden_size: 은닉층의 뉴런 수
- corpus: 단어 ID 목록
- window_size: 맥락의 크기

In [20]:
import sys
sys.path.append('..')
import numpy as np
from common.np import *
from common.layers import Embedding
from common.negative_sampling_layer import NegativeSamplingLoss


class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

### 4.3.2 CBOW 모델 학습 코드

In [ ]:
import sys
sys.path.append('..')
import numpy as np
from common import config
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

### 4.3.3 CBOW 모델 평가

In [ ]:
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle


pkl_file = 'coomon/cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)